In [1]:
#%load 'cube.py'
from cube import rubiks_cube
from cube import move_rubiks_cube

In [2]:
r_cube = rubiks_cube()  # In a initial_state 
r_cube

[[['F1', 'F2', 'F3'], ['F4', 'F5', 'F6'], ['F7', 'F8', 'F9']],
 [['B1', 'B2', 'B3'], ['B4', 'B5', 'B6'], ['B7', 'B8', 'B9']],
 [['D1', 'D2', 'D3'], ['D4', 'D5', 'D6'], ['D7', 'D8', 'D9']],
 [['L1', 'L2', 'L3'], ['L4', 'L5', 'L6'], ['L7', 'L8', 'L9']],
 [['R1', 'R2', 'R3'], ['R4', 'R5', 'R6'], ['R7', 'R8', 'R9']],
 [['U1', 'U2', 'U3'], ['U4', 'U5', 'U6'], ['U7', 'U8', 'U9']]]

In [5]:
cube_f = move_rubiks_cube(r_cube, 0) # FRONT
cube_f

[[['F7', 'F4', 'F1'], ['F8', 'F5', 'F2'], ['F9', 'F6', 'F3']],
 [['B1', 'B2', 'B3'], ['B4', 'B5', 'B6'], ['B7', 'B8', 'B9']],
 [['R7', 'R4', 'R1'], ['D4', 'D5', 'D6'], ['D7', 'D8', 'D9']],
 [['L1', 'L2', 'D1'], ['L4', 'L5', 'D2'], ['L7', 'L8', 'D3']],
 [['U7', 'R2', 'R3'], ['U8', 'R5', 'R6'], ['U9', 'R8', 'R9']],
 [['U1', 'U2', 'U3'], ['U4', 'U5', 'U6'], ['L9', 'L6', 'L3']]]

In [6]:
cube_1 = move_rubiks_cube(cube_f, 6)
cube_1

[[['F1', 'F2', 'F3'], ['F4', 'F5', 'F6'], ['F7', 'F8', 'F9']],
 [['B1', 'B2', 'B3'], ['B4', 'B5', 'B6'], ['B7', 'B8', 'B9']],
 [['D1', 'D2', 'D3'], ['D4', 'D5', 'D6'], ['D7', 'D8', 'D9']],
 [['L1', 'L2', 'L3'], ['L4', 'L5', 'L6'], ['L7', 'L8', 'L9']],
 [['R1', 'R2', 'R3'], ['R4', 'R5', 'R6'], ['R7', 'R8', 'R9']],
 [['U1', 'U2', 'U3'], ['U4', 'U5', 'U6'], ['U7', 'U8', 'U9']]]

In [7]:
def check_face_movement(face):
    # Basically it checks if has duplicate values in array
    initial_state = convert_in_vector(rubiks_cube())
    cube_x = convert_in_vector(move_rubiks_cube(rubiks_cube(), face))
    aux = 0
    try:
        for i in range(54):
            aux = cube_x[i]
            initial_state.remove(cube_x[i])
    except Exception, e:
        print "duplicado ", aux
        return False
    return True

def convert_in_vector(cube):
    # Converts the cube in a array
    values = []
    for face in cube:
        for row in face:
            for value in row:
                values.append(value)
    return values

In [8]:
[check_face_movement(i) for i in range(11)]

[True, True, True, True, True, True, True, True, True, True, True]

In [9]:
initial_state = convert_in_vector(rubiks_cube())
cube_x = convert_in_vector(r_cube)
aux = 0
try:
    for i in range(54):
        aux = cube_x[i]
        initial_state.remove(cube_x[i])
except Exception, e:
    print "duplicado ", aux
print "OK"

OK
